In [25]:
import plotly
plotly.tools.set_credentials_file(username='churtado', api_key='iaMRV6ydU9Ove5Yfy0R7')

In [1]:
%load_ext sparkmagic.magics

http://10.13.12.209:8998/

In [2]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

Added endpoint http://10.13.12.209:8998/
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,None,spark,idle,,,✔


SparkSession available as 'spark'.


In [3]:
%%spark
val sqlContext = new org.apache.spark.sql.hive.HiveContext(sc)

val df = sqlContext.read.format("com.crealytics.spark.excel").option("sheetName", "pc").option("useHeader", "true").option("inferSchema", "false").load("/user/hive/data/smmt/smmt.xls")
df.createTempView("pc")

sqlContext: org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@6fdd9f1
df: org.apache.spark.sql.DataFrame = [date: string, passenger_cars: string]


In [4]:
%%spark -c sql
SHOW TABLES

,database,tableName,isTemporary
0,default,cleaned_taxes,False
1,default,passenger_cars,False
2,,pc,True


In [5]:
%%spark -c sql
SELECT * from pc limit 3

,date,passenger_cars
0,2018-06-30,234945
1,2018-05-31,192649
2,2018-04-30,167911


In [6]:
%%spark -c sql
DROP TABLE IF EXISTS passenger_cars

""


In [7]:
%%spark -c sql 

CREATE TABLE passenger_cars AS
SELECT 
 date, 
 passenger_cars
FROM pc

""


In [35]:
%%spark -c sql -q -o df_passenger_cars
SELECT * from pc

Let's do a quick analysis of the data we have. There's only 2 variables, so this will be a time series forecast.

When decomposing a time series, there are 2 components to look out for:  
  
Systematic: Components of the time series that have consistency or recurrence and can be described and modeled.  
Non-Systematic: Components of the time series that cannot be directly modeled.  

Concretely, the components are:  
  
    Level: The average value in the series.  
    Trend: The increasing or decreasing value in the series.  
    Seasonality: The repeating short-term cycle in the series.  
    Noise: The random variation in the series.  
  
First, let's look at the data:

Let's try to decompose it. First set up our dataframe

In [36]:
%matplotlib inline

import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

df_passenger_cars_decomp = df_passenger_cars
df_passenger_cars_decomp['date'] = pd.to_datetime(df_passenger_cars_decomp['date'])
df_passenger_cars_decomp = df_passenger_cars_decomp.set_index('date')

df_passenger_cars_decomp.head(3)

,date,passenger_cars
0,2018-06-30,234945
1,2018-05-31,192649
2,2018-04-30,167911


We're looking at this breakdown assuming an additive model first:

In [75]:
from statsmodels.tsa.seasonal import seasonal_decompose

result_additive = seasonal_decompose(df_passenger_cars_decomp, model='additive')
# print(result_additive.trend)
# print(result_additive.seasonal)
# print(result_additive.resid)
# print(result_additive.observed)

In [119]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Scatter(
    x = result_additive.trend.index,
    y = result_additive.trend.values,
    xaxis='x3',
    yaxis='y3',
    name='trend'
)
trace2 = go.Scatter(
    x = result_additive.seasonal.index,
    y = result_additive.seasonal.values,
    xaxis='x2',
    yaxis='y2',
    name='seasonal'
)
trace3 = go.Scatter(
    x = result_additive.resid.index,
    y = result_additive.resid.values,
    xaxis='x',
    yaxis='y',
    name='residual'
)
trace4 = go.Scatter(
    x = result_additive.observed.index,
    y = result_additive.observed.values,
    xaxis='x4',
    yaxis='y4',
    name='observed'
)
data = [trace4, trace1, trace2, trace3]
layout = go.Layout(
    xaxis=dict(
        domain=[0, 1]
    ),
    yaxis=dict(
        domain=[0, 0.23]
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='x2'
    ),
    yaxis2=dict(
        domain=[0.26, 0.48],
        anchor='y2'
    ),
    xaxis3=dict(
        domain=[0, 1],
        anchor='y3'
    ),
    yaxis3=dict(
        domain=[0.52, 0.73]
    ),
    xaxis4=dict(
        domain=[0, 1],
        anchor='y4'
    ),
    yaxis4=dict(
        domain=[0.76, 1],
        anchor='x4'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='additive-subplots')

In [79]:
from statsmodels.tsa.seasonal import seasonal_decompose

result_mult = seasonal_decompose(df_passenger_cars_decomp, model='multiplicative')
# print(result_additive.trend)
# print(result_additive.seasonal)
# print(result_additive.resid)
# print(result_additive.observed)

In [120]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Scatter(
    x = result_mult.trend.index,
    y = result_mult.trend.values,
    xaxis='x3',
    yaxis='y3',
    name='trend'
)
trace2 = go.Scatter(
    x = result_mult.seasonal.index,
    y = result_mult.seasonal.values,
    xaxis='x2',
    yaxis='y2',
    name='seasonal'
)
trace3 = go.Scatter(
    x = result_mult.resid.index,
    y = result_mult.resid.values,
    xaxis='x',
    yaxis='y',
    name='residual'
)
trace4 = go.Scatter(
    x = result_mult.observed.index,
    y = result_mult.observed.values,
    xaxis='x4',
    yaxis='y4',
    name='observed'
)
data = [trace4, trace1, trace2, trace3]
layout = go.Layout(
    xaxis=dict(
        domain=[0, 1]
    ),
    yaxis=dict(
        domain=[0, 0.23]
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='x2'
    ),
    yaxis2=dict(
        domain=[0.26, 0.48],
        anchor='y2'
    ),
    xaxis3=dict(
        domain=[0, 1],
        anchor='y3'
    ),
    yaxis3=dict(
        domain=[0.52, 0.73]
    ),
    xaxis4=dict(
        domain=[0, 1],
        anchor='y4'
    ),
    yaxis4=dict(
        domain=[0.76, 1],
        anchor='x4'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='multiplicative-subplots')

  
We're going to try and do a linear regression on this
  